In [3]:
import minsearch
import json

In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc["course"] = course_dict["course"]
        documents.append(doc)

In [8]:
from dotenv import load_dotenv
from os import getenv

load_dotenv()

True

In [23]:
from groq import Groq

In [27]:
def fit_index():
    index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
    )

    index.fit(documents)

    return index

def search(query):
    index = fit_index()

    boost = {
        'question': 3.0,
        'section': 0.5
    }

    results = index.search(
        query=query,
        filter_dict={"course": "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5
    )

    return results

def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE.

    QUESTION: {question}

    CONTEXT: {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

def llm(prompt):
    client = Groq(api_key=getenv("GROQ_API_KEY"))
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-8b-8192"
    )

    return response.choices[0].message.content

In [25]:
query = "how do i run kafka?"
def rag(query):
    search_results = search( query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

rag(query)

"Based on the context, I see that there are questions related to running Kafka-based code in Python and Java. Since you are asking how to run Kafka in general, I will answer that your context doesn't cover a specific way to run Kafka. However, I can suggest a few ways to run Kafka streams:\n\nFrom the context, we can see that there's a way to run a Java Kafka producer/consumer/kstreams/etc. in the terminal by running the provided Java command:\n\n`java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`\n\nAdditionally, it seems that Kafka is being used in a Python context, specifically in the 'Module 6: streaming with kafka' section. However, running Kafka itself is not explicitly covered in the provided context. If you provide more information on what you want to achieve or which specific component of Kafka you want to run, I may be able to provide more tailored advice."

In [30]:
from elasticsearch import Elasticsearch

In [31]:
es_client = Elasticsearch('http://localhost:9200')

In [35]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [36]:
from tqdm.auto import tqdm

c:\Users\sebas\miniconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:48<00:00, 19.60it/s]


In [52]:
def es_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }


    response = es_client.search(index=index_name, body=search_query)

    results_docs = []
    for hit in response['hits']['hits']:
        results_docs.append(hit['_source'])

    return results_docs

In [58]:
query = "I just discovered the course. Can I still join?"

In [59]:
es_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and 

In [60]:
def rag(query):
    search_results = es_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

rag(query)

"ANSWER: Yes, even if you don't register, you're still eligible to submit the homeworks."